# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [78]:
# Your solution goes here
import pandas as pd
from bs4 import BeautifulSoup
import requests

def get_soup():
    print('Película: ')
    from_to_date = '1990-01-01,1992-12-31' #input('De qué fecha a qué fecha quiere buscar. formato (año-mes-dia,año-mes-dia) --> ')
    from_to_rating = '7.5/10 '#input('De qué rango de valoraciones quieres filtrar las pelis, de menor a mayor. formato (num,num) ej(7.5,10) --> ')
    url = f'https://www.imdb.com/search/title/?title_type=feature&release_date={from_to_date}&user_rating={from_to_rating}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    pelicula_list = []
    for index, pelicula in enumerate(peliculas.find_all('li',class_='ipc-metadata-list-summary-item')):
        name = " ".join(pelicula.find('h3', class_='ipc-title__text').text.split()[1:])
        index = index+1
        rating, votes = pelicula.find('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').text.split()
        rating = float(rating)
        votes = votes.strip('()')
        year, duration, rate_age = pelicula.find_all('span', class_='sc-b189961a-8 kLaxqf dli-title-metadata-item')
        year = int(year.text.strip())
        duration = duration.text.strip()
        rate_age = rate_age.text.strip()
        # Gender -> Genero no se puede sacar via WebScrapping
        description = pelicula.find('div', class_='ipc-html-content-inner-div').text.strip()
        # Director -> Tampoco se puede obtener el nombre de Director
        # Gross -> Tampoco se puede obtener el beneficio de la película
        pelicula_list.append([name, year, duration, rate_age, rating, votes, description ])
    df = pd.DataFrame(pelicula_list, columns=[''])
    return df
df_peliculas = get_soup()

Película: 


In [81]:

df_peliculas.index = df

,0,1,2,3,4,5,6,7
0,,,,,,,,
1,1,Presunto inocente,1990,2h 7m,13,6.9,47K,As a lawyer investigates the murder of a colle...
2,2,El silencio de los corderos,1991,1h 58m,18,8.6,1.6M,A young F.B.I. cadet must receive the help of ...
3,3,Uno de los nuestros,1990,2h 25m,18,8.7,1.3M,The story of Henry Hill and his life in the ma...
4,4,Bailando con lobos,1990,3h 1m,A,8.0,294K,"Lieutenant John Dunbar, assigned to a remote w..."
5,5,Terminator 2: El juicio final,1991,2h 17m,18,8.6,1.2M,"A cyborg, identical to the one who failed to k..."
6,6,Drácula de Bram Stoker,1992,2h 8m,18,7.4,241K,The centuries old vampire Count Dracula comes ...
7,7,Instinto básico,1992,2h 7m,18,7.1,221K,A violent police detective investigates a brut...
8,8,Algunos hombres buenos,1992,2h 18m,13,7.7,290K,"Lt. Daniel Kaffee, a US military lawyer, defen..."
9,9,Ellas dan el golpe,1992,2h 8m,A,7.3,122K,"During World War II, sisters Dottie and Kit jo..."




## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here